# Survey data 
Notes:
- for binary 'yes' or 'no' variables, 0 is assumed to be 'no', unless otherwise stated
- numbering of survey doc is inconsistent in some places
- there are questions in the survey which do not have variables/data
    - 2b. Are any of these a closer description to your gender identity?
    - 19a: Are there any medications that a doctor said you should be taking that, for whatever reason, you are not taking? 
- 18 participants identified as unhoused in the leads assessment but 19 are responding to the final 2 questions for unhoused participants

***

### Import libraries and data

In [510]:
import pandas as pd
import numpy as np
import statistics

file = 'APTHomelessnessDeidentified data set- Quantitative.xlsx'
hdf = pd.read_excel(file,header=1)

### Helper functions

In [511]:
def binary_helper(var_name,axis_name,df):
    qdata = pd.DataFrame(df.value_counts().astype(int)).set_axis(['N'], axis=1)
    incl_values = qdata.index
    qdata.index = [x[0] for x in incl_values]
    qdata['%'] = ((qdata/np.sum(qdata['N'].values))*100).round(3)
    qdata.loc['Total'] = [np.sum(qdata['N'].values), np.sum(qdata['%'])]
    qdata['N'] = qdata['N'].values.astype(int)
    qdata.index.name = axis_name
    return qdata

def binary_stat(var_name,axis_name,df):
    q = df[[var_name]].replace({0:'no', 1:'yes'}).set_axis([axis_name], axis=1)
    qdata = binary_helper(var_name, axis_name, q)
    return qdata

def binary_stat2(var_name,axis_name,df):
    q = df[[var_name]].replace({1:'yes', 2:'no'}).set_axis([axis_name], axis=1)
    qdata = binary_helper(var_name, axis_name, q)
    return qdata

def binary_stat3(var_name,axis_name,df):
    q = df[[var_name]].replace({1:'yes', 2:'no', 3:'does not apply'}).set_axis([axis_name], axis=1)
    qdata = binary_helper(var_name, axis_name, q)
    return qdata

def times_count(var_name, df):
    qdata = pd.DataFrame(df[var_name].describe()).round(2).rename(columns={var_name: ' '}, index={'count':'N (responses)'})
    display(qdata)
    print('mode: '+ str(df[var_name].mode()[0]))

def multi_column(columns, repl_names, df):
    q = df[columns].set_axis(repl_names, axis=1)
    q = q[~(q==0).all(axis=1)]
    qdata = pd.DataFrame(q.idxmax(axis=1).value_counts().astype(int)).set_axis(['N'], axis=1)
    qdata['%'] = ((qdata/np.sum(qdata.values))*100).round(10)
    qdata.loc['Total'] = [np.sum(qdata['N']), np.sum(qdata['%'])]
    qdata['%'] = qdata['%'].round(3)
    qdata['N'] = qdata['N'].values.astype(int)
    return qdata

def single_cat_var(column, var_dict, header, df):
    q = df[[column]].set_axis([header], axis=1)
    q = df[[column]].replace(var_dict)
    qdata = pd.DataFrame(q.value_counts().astype(int)).set_axis(['N'], axis=1)
    qdata['%'] = (qdata/np.sum(qdata['N'].values))*100
    incl_values = qdata.index
    qdata.index = [x[0] for x in incl_values]
    for val in var_dict.values():
        if val not in incl_values:
            qdata.loc[val] = [0, 0.0]
    qdata.loc['Total'] = [np.sum(qdata['N'].values), np.sum(qdata['%'])]
    qdata['N'] = qdata['N'].values.astype(int)
    qdata.index.name = header
    return qdata

***

# Section 1: Demographics Survey Data

### Question 1: What is your housing situation today? <br>
current_housing___1 = I do not have housing (I am staying with others, in a hotel, in a shelter, living outside on the street, on a beach, in a car, abandoned building, bus or train station, or in a park) <br>
current_housing___2 = I have housing today, but I am worried about losing housing in the future. <br>
current_housing___3 = I have housing 

In [500]:
columns = ['current_housing___1','current_housing___2','current_housing___3']
repl_names = ['Do not have housing', 'Have housing but worried', 'I have housing']
q1 = multi_column(columns, repl_names, hdf)
display(q1)

,N,%
Do not have housing,30,62.500
Have housing but worried,14,29.167
I have housing,4,8.333
Total,48,100.000


### Question 2: Altogether, how much of your life have you been homeless—would you say less than a week, more than a week but less than a month, more than a month but less than a year or more than a year?
duration_homelessness___1 = less than a week <br>
duration_homelessness___2 = more than a week but less than a month<br>
duration_homelessness___3 = more than a month but less than a year<br>
duration_homelessness___4 = more than a year

In [501]:
columns = ['duration_homelessness___1','duration_homelessness___2','duration_homelessness___3','duration_homelessness___4']
repl_names = ['less than a week', 'more than a week but less than a month', 'more than a month but less than a year', 'more than a year']
q22 = multi_column(columns, repl_names, hdf)
display(q22)

,N,%
more than a year,40,81.633
more than a month but less than a year,8,16.327
more than a week but less than a month,1,2.041
Total,49,100.000


In [ ]:
def multi_column(columns, repl_names, df):
    q = df[columns].set_axis(repl_names, axis=1)
    q = q[~(q==0).all(axis=1)]
    qdata = pd.DataFrame(q.idxmax(axis=1).value_counts().astype(int)).set_axis(['N'], axis=1)
    qdata['%'] = ((qdata/np.sum(qdata.values))*100).round(10)
    qdata.loc['Total'] = [np.sum(qdata['N']), np.sum(qdata['%'])]
    qdata['%'] = qdata['%'].round(3)
    qdata['N'] = qdata['N'].values.astype(int)
    return qdata

In [ ]:
def multi_column2(columns, repl_names, df):
    q = df[columns].set_axis(repl_names, axis=1)
    q = q[~(q==0).all(axis=1)]
    return q

In [507]:
hdf.groupby(['current_housing___1']).count()

,record_id,form_1_complete,current_housing___2,current_housing___3,duration_homelessness___1,duration_homelessness___2,duration_homelessness___3,duration_homelessness___4,biological_sex,gender_identity,...,in_the_last_12_months_didyes,problems_with_childcare_le,past_12_months_unable_to_s,past_12_months_healthcare,help_reading_hospital_mate,lack_of_companionship,is_patient_currently_house,if_yes_to_housed_past_12_m,are_you_worried_that_in_th,health_leads_screening_toolkit_complete
current_housing___1,,,,,,,,,,,,,,,,,,,,,
0,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,19,19,20
1,30,30,30,30,30,30,30,30,30,30,...,30,30,30,29,30,30,30,0,0,30


### Question 2: What was your biological sex assigned at birth? 
1 = Female <br>
2 = Male<br>
3 = Intersex<br>
4 = None of these describe me (optional free text)<br>
5 = Prefer not to answer<br><br>
(single categorical variable)

In [405]:
column = 'biological_sex'
var_dict = {1:'female', 2:'male',3:'intersex',4:'none',5:'prefer not to answer'}
header = 'sex assigned at birth'
q2 = single_cat_var(column, var_dict, header, hdf)
q2

,N,%
sex assigned at birth,,
male,31,62.0
female,19,38.0
intersex,0,0.0
none,0,0.0
prefer not to answer,0,0.0
Total,50,100.0


### Question 2a: What terms best express how you describe your gender identity? (Check all that apply)
1 = Man <br>
2 = Woman<br>
3 = Non-binary<br>
4 = Transgender<br>
5 = None of these describe me, and I’d like to consider additional options<br>
6 = Prefer not to answer<br><br>
(single categorical variable)

In [406]:
column = 'gender_identity'
var_dict = {1:'man', 2:'woman',3:'non-binary',4:'transgender',5:'none of these describe me', 6:'prefer not to answer'}
header = 'gender identity'
q2a = single_cat_var(column, var_dict, header, hdf)
q2a

,N,%
gender identity,,
man,31,62.0
woman,19,38.0
non-binary,0,0.0
transgender,0,0.0
none of these describe me,0,0.0
prefer not to answer,0,0.0
Total,50,100.0


### Question 3: How old are you?

In [407]:
q3 = pd.DataFrame(hdf['age'].describe()).round(2).rename(columns={'age': ''}, index={'count':'N'})
q3

,
N,50.00
mean,43.68
std,9.87
min,25.00
25%,37.00
50%,43.50
75%,50.50
max,61.00


### Question 4: Are you of Hispanic, Latino, or Spanish origin?

In [408]:
var_name = 'hispanic_latino___1'
axis_name = 'hispanic, latino or spanish origin?'
q4 = binary_stat(var_name,axis_name,hdf)
q4

,N,%
"hispanic, latino or spanish origin?",,
no,42,84.0
yes,8,16.0
Total,50,100.0


**If yes, of what origin?**

In [409]:
q4a = pd.DataFrame(hdf['if_hispanic_latino_or_span'])
q4a[~q4a.isnull().any(axis=1)].reset_index(drop=True)

,if_hispanic_latino_or_span
0,Dominican
1,Puerto Rican
2,spanish
3,Dominican
4,Purto rican
5,Puerto rican
6,Peurto Rican
7,Puerto Rican


### Question 5: What is your race?
what_is_your_race___1: White<br>
what_is_your_race___2: Black<br>
what_is_your_race___3: American Indian<br>
what_is_your_race___4: Chinese<br>
what_is_your_race___5: Filipino<br>
what_is_your_race___6: Asian Indian<br>
what_is_your_race___7: Vietnamese<br>
what_is_your_race___8: Korean<br>
what_is_your_race___9: Japanese<br>
1what_is_your_race___10: Other Asian (Pakistani)<br>
1what_is_your_race___11: Native Hawaiian<br>
1what_is_your_race___12: Other

In [410]:
columns = ['what_is_your_race___1','what_is_your_race___2','what_is_your_race___3','what_is_your_race___4', 'what_is_your_race___5', 'what_is_your_race___6',
           'what_is_your_race___7', 'what_is_your_race___8', 'what_is_your_race___9', 'what_is_your_race___10', 'what_is_your_race___11', 'what_is_your_race___12']
repl_names = ['white','black','american indian', 'chinese', 'filipino', 'asian indian', 'vietnamese', 'korean', 'japanese','other asian (pakistani)','native hawaiian','other']
q6 = multi_column(columns, repl_names, hdf)
display(q6)

,N,%
white,38,77.551
black,8,16.327
other,3,6.122
Total,49,100.000


**Additional race information provided in free text**

In [411]:
q5b = pd.DataFrame(hdf['race_origin'])
q5b[~q5b.isnull().any(axis=1)].reset_index(drop=True)

,race_origin
0,irish
1,"scottish, german"
2,African American
3,Irish and italian
4,Italian
5,caucasian
6,African American
7,just white/ NA
8,english/irish
9,English


### Question 7: Marital status
marital_status___1: married
<br> marital_status___2: divorced
<br> marital_status___3: widowed
<br> marital_status___4: separated
<br> marital_status___5: never married

In [412]:
columns = ['marital_status___1','marital_status___2','marital_status___3', 'marital_status___4', 'marital_status___5']
repl_names = ['married', 'divorced', 'widowed', 'separated', 'never married']
q7 = multi_column(columns, repl_names, hdf)
display(q7)

,N,%
never married,31,63.265
divorced,9,18.367
separated,6,12.245
married,3,6.122
Total,49,100.000


### Question 8: We would like to know about what you do-are you working now, looking for work, retired, keeping house, a student, or what? 
work_status___1: Working now
<br>work_status___2: Only Temporarily laid off
<br>work_status___3: Retired
<br>work_status___4: Disabled
<br>work_status___5: Keeping house
<br>work_status___6: Student
<br>work_status___7: Other
<br>work_status___unemployed: Unemployed, looking for work (separate var column)

In [413]:
columns = ['work_status___1','work_status___2', 'work_status___unemployed', 'work_status___3', 'work_status___4', 'work_status___5', 'work_status___6','work_status___7']
repl_names = ['working now', 'temporarily laid off', 'unemployed, looking', 'retired', 'disabled', 'keeping house', 'student', 'other']
q8 = multi_column(columns, repl_names, hdf)
display(q8)

,N,%
"unemployed, looking",28,57.143
disabled,12,24.490
working now,5,10.204
other,3,6.122
keeping house,1,2.041
Total,49,100.000


***

# Section 2: Patients currently experiencing homelessness

## 2A: History of Housing and Homelessness 

### Question 1: Where do you sleep most frequently?
1: Shelters
<br>2: Transitional Housing
<br>3: Safe Haven
<br>4: Outdoors
<br>5: Other (Specify)

In [414]:
column = 'where_do_you_sleep_most_fr'
var_dict = {1:'shelters', 2:'transitional housing',3:'safe haven',4:'outdoors',5:'other'}
header = 'where do you sleep most often'
hq1 = single_cat_var(column, var_dict, header, hdf)
hq1

,N,%
where do you sleep most often,,
other,17,53.125
outdoors,9,28.125
shelters,4,12.500
transitional housing,2,6.250
safe haven,0,0.000
Total,32,100.000


### Question 2: How long has it been since you lived in permanent stable housing (months)? 

In [415]:
hq2 = pd.DataFrame(hdf['time_permanenthousing'].describe()).round(2).rename(columns={'time_permanenthousing': 'months since permanent housing'}, index={'count':'N (responses)'})
hq2

,months since permanent housing
N (responses),31.00
mean,61.26
std,87.45
min,1.00
25%,6.50
50%,24.00
75%,66.00
max,360.00


### Question 3: In the last three years, how many times have you been homeless? 

In [416]:
hq3 = pd.DataFrame(hdf['no_of_times_homeless_three_yrs'].describe()).round(2).rename(columns={'no_of_times_homeless_three_yrs': 'number of times homeless past three years'}, index={'count':'N (responses)'})
hq3
# data check

,number of times homeless past three years
N (responses),31.00
mean,2.42
std,3.39
min,1.00
25%,1.00
50%,2.00
75%,2.50
max,20.00


### Question 3: In the last three years, how many times have you been homeless? 

In [417]:
hq3 = pd.DataFrame(hdf['no_of_times_homeless_three_yrs'].describe()).round(2).rename(columns={'no_of_times_homeless_three_yrs': 'number of times homeless past three years'}, index={'count':'N (responses)'})
hq3
# data check

,number of times homeless past three years
N (responses),31.00
mean,2.42
std,3.39
min,1.00
25%,1.00
50%,2.00
75%,2.50
max,20.00


## 2B: Risks

### Question 4: In the last 6 months how many times have you: 

**Received care at an emergency department?**

In [418]:
hq4a = pd.DataFrame(hdf['a_received_health_care_at'].describe()).round(2).rename(columns={'a_received_health_care_at': ' '}, index={'count':'N (responses)'})
display(hq4a)
print('mode: '+ str(hdf['a_received_health_care_at'].mode()[0]))

,
N (responses),31.00
mean,1.74
std,2.46
min,0.00
25%,0.00
50%,1.00
75%,3.00
max,10.00


mode: 0.0


**Taken an ambulance to hospital?**

In [419]:
hq4b = pd.DataFrame(hdf['b_taken_an_ambulance_to_th'].describe()).round(2).rename(columns={'b_taken_an_ambulance_to_th': ' '}, index={'count':'N (responses)'})
display(hq4b)
print('mode: '+ str(hdf['b_taken_an_ambulance_to_th'].mode()[0]))

,
N (responses),31.00
mean,1.52
std,2.13
min,0.00
25%,0.00
50%,1.00
75%,2.00
max,10.00


mode: 0.0


**Been hospitalized as an inpatient?**

In [420]:
hq4c = pd.DataFrame(hdf['c_been_hospitalized_as_an'].describe()).round(2).rename(columns={'c_been_hospitalized_as_an': ' '}, index={'count':'N (responses)'})
display(hq4c)
print('mode: '+ str(hdf['c_been_hospitalized_as_an'].mode()[0]))

,
N (responses),31.00
mean,0.74
std,1.26
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,6.00


mode: 0.0


**Used a crisis service, including sexual assault crisis, mental health crisis, family/intimate violence, distress centers and suicide prevention hotlines?**

In [421]:
hq4d = pd.DataFrame(hdf['d_used_a_crisis_service_in'].describe()).round(2).rename(columns={'d_used_a_crisis_service_in': ' '}, index={'count':'N (responses)'})
display(hq4d)
print('mode: '+ str(hdf['d_used_a_crisis_service_in'].mode()[0]))

,
N (responses),31.00
mean,0.32
std,0.83
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,4.00


mode: 0.0


**Talked to police because you witnessed a crime, were the victim of a crime, or the alleged perpetrator of a crime or because the police told you that you must move along?**

In [422]:
hq4e = pd.DataFrame(hdf['e_talked_to_police_because'].describe()).round(2).rename(columns={'e_talked_to_police_because': ' '}, index={'count':'N (responses)'})
display(hq4e)
print('mode: '+ str(hdf['e_talked_to_police_because'].mode()[0]))

,
N (responses),31.00
mean,0.58
std,1.03
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,4.00


mode: 0.0


**Stayed one or more nights in a holding cell, jail or prison, whether that was a short-term stay like the drunk tank, a longer stay for a more serious offence, or anything in between?**

In [423]:
hq4f = pd.DataFrame(hdf['f_stayed_one_or_more_night'].describe()).round(2).rename(columns={'f_stayed_one_or_more_night': ' '}, index={'count':'N (responses)'})
display(hq4f)
print('mode: '+ str(hdf['f_stayed_one_or_more_night'].mode()[0]))

,
N (responses),31.00
mean,0.48
std,1.81
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,10.00


mode: 0.0


### Question 5: Have you been attacked or beaten up since you have become homeless?

In [424]:
var_name = 'have_you_been_attacked_or'
axis_name = 'Attacked/beaten?'
hq5 = binary_stat(var_name,axis_name,hdf)
hq5

,N,%
Attacked/beaten?,,
no,15,50.0
yes,15,50.0
Total,30,100.0


### Question 6: Have you threatened to or tried to harm yourself or anyone else in the last year?

In [425]:
var_name = 'have_you_threatened_to_or'
axis_name = 'Harm yourself or anyone else?'
hq6 = binary_stat(var_name,axis_name,hdf)
hq6

,N,%
Harm yourself or anyone else?,,
no,26,83.871
yes,5,16.129
Total,31,100.000


### Question 7: Do you have any legal stuff going on right now that may result in you being locked up, having to pay fines, or that make it more difficult to rent a place to live? 

In [426]:
var_name = 'do_you_have_any_legal_stuf'
axis_name = 'Current legal issues?'
hq7 = binary_stat(var_name,axis_name,hdf)
hq7

,N,%
Current legal issues?,,
no,22,70.968
yes,9,29.032
Total,31,100.000


### Question 8: Does anybody force or trick you to do things that you do not want to do?

In [427]:
var_name = 'does_anybody_force_or_tric'
axis_name = 'Forced or tricked?'
hq8 = binary_stat(var_name,axis_name,hdf)
hq8

,N,%
Forced or tricked?,,
no,27,87.097
yes,4,12.903
Total,31,100.000


### Question 9: Do you ever do things that may be considered to be risky like exchange sex for money, run drugs for someone, have unprotected sex with someone you don’t know, share a needle, or anything like that? 

In [428]:
var_name = 'risky_things'
axis_name = 'Risky activity'
hq9 = binary_stat(var_name,axis_name,hdf)
hq9

,N,%
Risky activity,,
no,20,64.516
yes,11,35.484
Total,31,100.000


## 2C: Socialization & Daily Functioning 

### Question 10: Is there any person, past landlord, business, bookie, dealer, or government group like the IRS that thinks you owe them money? 

In [429]:
var_name = 'is_there_any_person_past_l'
axis_name = 'Owe money'
hq10 = binary_stat(var_name,axis_name,hdf)
hq10

,N,%
Owe money,,
no,24,77.419
yes,7,22.581
Total,31,100.000


### Queation 11: Do you get any money from the government, a pension, an inheritance, working under the table, a regular job, or anything like that? 

In [430]:
var_name = 'do_you_get_any_money_from'
axis_name = 'Money received'
hq11 = binary_stat(var_name,axis_name,hdf)
hq11


,N,%
Money received,,
no,26,83.871
yes,5,16.129
Total,31,100.000


### Question 12: Do you have planned activities, other than just surviving, that make you feel happy and fulfilled? 

In [431]:
var_name = 'do_you_have_planned_activi'
axis_name = 'Fulfilling activities'
hq12 = binary_stat(var_name,axis_name,hdf)
hq12

,N,%
Fulfilling activities,,
no,16,51.613
yes,15,48.387
Total,31,100.000


### Question 12a: If yes, what are the activities? 

In [432]:
hq12a = pd.DataFrame(hdf['if_yes_what_are_the_activi']).set_axis(['activities'], axis=1)
hq12a[~hq12a.isnull().any(axis=1)]

,activities
3,love to watch movies and games related to history
4,drawing
8,hiking and softball
9,Bingo
18,Music classes
20,taking care of mother and dog
26,"gardening, harvesting, cooking, feeding others"
27,Coin collection
31,Looking for work
35,Planning on becoming a recovery coach


### Question 13: Are you currently able to take care of basic needs like bathing, changing clothes, using a restroom, getting food and clean water and other things like that? 

In [433]:
var_name = 'are_you_currently_able_to'
axis_name = 'Take care of basic needs'
hq13 = binary_stat(var_name,axis_name,hdf)
hq13

,N,%
Take care of basic needs,,
yes,25,80.645
no,6,19.355
Total,31,100.000


### Question 14: Is your current homelessness in any way caused by a relationship that broke down, an unhealthy or abusive relationship, or because family or friends caused you to become evicted? 

In [434]:
var_name = 'current_homelessness_cause'
axis_name = 'Related to relationship'
hq14 = binary_stat(var_name,axis_name,hdf)
hq14


,N,%
Related to relationship,,
no,16,51.613
yes,15,48.387
Total,31,100.000


## 2D: Wellness 

### Question 15: Have you ever had to leave an apartment, shelter program, or other place you were staying because of your physical health? 

In [435]:
var_name = 'have_you_ever_had_to_leave'
axis_name = 'Had to leave'
hq15 = binary_stat(var_name,axis_name,hdf)
hq15

,N,%
Had to leave,,
no,26,83.871
yes,5,16.129
Total,31,100.000


### Question 16: Do you have any chronic health issues with your liver, kidneys, stomach, lungs or heart?

In [436]:
var_name = 'do_you_have_any_chronic_he'
axis_name = 'Chronic health conditions'
hq16 = binary_stat(var_name,axis_name,hdf)
hq16

,N,%
Chronic health conditions,,
no,19,61.29
yes,12,38.71
Total,31,100.00


### Question 17: When you are sick or not feeling well, do you avoid getting help? 

In [437]:
var_name = 'when_you_are_sick_or_not_f'
axis_name = 'Avoid getting help'
hq17 = binary_stat(var_name,axis_name,hdf)
hq17


,N,%
Avoid getting help,,
yes,16,51.613
no,15,48.387
Total,31,100.000


### Question 18: Has drinking or drug use led you to being kicked out of an apartment or program where you were staying in the past? 

In [438]:
var_name = 'has_drinking_or_drug_use_l'
axis_name = 'Substances / housing issues'
hq18 = binary_stat(var_name,axis_name,hdf)
hq18

,N,%
Substances / housing issues,,
yes,18,58.065
no,13,41.935
Total,31,100.000


### Question 19: Are there any medications that a doctor said you should be taking that, for whatever reason, you are not taking? 

In [439]:
var_name = 'are_there_any_medications'
axis_name = 'Not taking medications'
hq19 = binary_stat(var_name,axis_name,hdf)
hq19

,N,%
Not taking medications,,
no,17,54.839
yes,14,45.161
Total,31,100.000


### Question 20: Are there any medications like painkillers that you don’t take the way the doctor prescribed or where you sell the medication? 

In [440]:
var_name = 'medications_abuse'
axis_name = 'Medication misuse'
hq20 = binary_stat(var_name,axis_name,hdf)
hq20

,N,%
Medication misuse,,
no,26,83.871
yes,5,16.129
Total,31,100.000


### Question 21: Has your current period of homelessness been caused by an experience of emotional, physical, psychological, sexual, or other type of abuse, or by any other trauma you have experienced?

In [441]:
var_name = 'has_your_current_period_of'
axis_name = 'Abuse/trauma'
hq21 = binary_stat(var_name,axis_name,hdf)
hq21

,N,%
Abuse/trauma,,
no,17,54.839
yes,14,45.161
Total,31,100.000


***

# Section 3: Patients with a past homelessness experience who are currently housed

## 3A. History of Housing and Homelessness 

### Question 1: When you were experiencing homelessness while coming to APT, where did you sleep most frequently?
1: Shelters
<br>2: Transitional Housing
<br>3: Safe Haven
<br>4: Outdoors
<br>5: Other (Specify)

In [442]:
column = 'when_you_were_experiencing'
var_dict = {1:'shelters', 2:'transitional housing',3:'safe haven',4:'outdoors',5:'other'}
header = 'where do you sleep most often'
phq1 = single_cat_var(column, var_dict, header, hdf)
phq1

,N,%
where do you sleep most often,,
other,9,47.368421
outdoors,8,42.105263
shelters,1,5.263158
transitional housing,1,5.263158
safe haven,0,0.000000
Total,19,100.000000


### Question 2: How long did you live without permanent stable housing? 

In [443]:
phq2 = pd.DataFrame(hdf['hx_homeless_permanent_housing'].describe()).round(2).rename(columns={'hx_homeless_permanent_housing': 'months without permanent housing'}, index={'count':'N (responses)'})
phq2

,months without permanent housing
N (responses),18.00
mean,56.64
std,81.84
min,1.50
25%,11.25
50%,21.00
75%,57.00
max,324.00


### Question 3: In the last three years, how many times have you been homeless? 

In [444]:
phq3 = pd.DataFrame(hdf['in_the_last_three_years_ho'].describe()).round(2).rename(columns={'in_the_last_three_years_ho': 'times homeless 3 years'}, index={'count':'N (responses)'})
phq3

,times homeless 3 years
N (responses),18.00
mean,7.22
std,23.17
min,1.00
25%,1.00
50%,2.00
75%,2.00
max,100.00


### Question 4: Now, where do you sleep most frequently?
1: Own house or apartment
<br>2: Friend/family house or apartment
<br>3: Shelters
<br>4: Transitional Housing
<br>5: Safe Haven
<br>6: Outdoors
<br>7: Other (Specify)

In [445]:
columns = ['now_where_do_you_sleep_mos___1','now_where_do_you_sleep_mos___2','now_where_do_you_sleep_mos___3','now_where_do_you_sleep_mos___4',
           'now_where_do_you_sleep_mos___5','now_where_do_you_sleep_mos___6','now_where_do_you_sleep_mos___7']
repl_names = ['own house or apt', 'friend/family house or apt', 'shelters', 'transitional housing', 'safe haven', 'outdoors', 'other']
phq4 = multi_column(columns, repl_names, hdf)
display(phq4)

,N,%
friend/family house or apt,7,36.842
own house or apt,5,26.316
transitional housing,5,26.316
shelters,1,5.263
other,1,5.263
Total,19,100.000


## 3B: Risks

### 4. In the last six months, how many times have you:

**Received health care at an emergency department/room?**

In [446]:
var_name = 'whilehomeless_received_health_care_at'
times_count(var_name,hdf)

,
N (responses),19.00
mean,1.53
std,2.91
min,0.00
25%,0.00
50%,1.00
75%,2.00
max,13.00


mode: 0.0


**Taken an ambulance to the hospital?**

In [447]:
var_name = 'whilehomeless_taken_an_ambulance_to_th'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.89
std,1.63
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,5.00


mode: 0.0


**Been hospitalized as an inpatient?**

In [448]:
var_name = 'homeless_hospitalized'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.53
std,0.90
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,3.00


mode: 0.0


**Used a crisis service, including sexual assault crisis, mental health crisis, family/intimate violence, distress centers and suicide prevention hotlines?**

In [449]:
var_name = 'while_homeless_crisesservice'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.37
std,0.68
min,0.00
25%,0.00
50%,0.00
75%,0.50
max,2.00


mode: 0.0


**Used a crisis service, including sexual assault crisis, mental health crisis, family/intimate violence, distress centers and suicide prevention hotlines?**

In [450]:
var_name = 'while_homeless_crisesservice'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.37
std,0.68
min,0.00
25%,0.00
50%,0.00
75%,0.50
max,2.00


mode: 0.0


**Talked to police because you witnessed a crime, were the victim of a crime, or the alleged perpetrator of a crime or because the police told you that you must move along?**

In [451]:
var_name = 'homeless_talked_to_police'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.42
std,1.02
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,3.00


mode: 0.0


**Stayed one or more nights in a holding cell, jail or prison, whether that was a short-term stay like the drunk tank, a longer stay for a more serious offence, or anything in between?**

In [452]:
var_name = 'homeless_stayed_one_or_more_night'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.26
std,0.56
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,2.00


mode: 0.0


### Question 5: Were you ever attacked or beaten up while you were homeless? 

In [453]:
var_name = 'were_you_ever_attacked_or'
axis_name = 'Attacked'
phq5 = binary_stat(var_name,axis_name,hdf)
phq5

,N,%
Attacked,,
no,11,57.895
yes,8,42.105
Total,19,100.000


### Question 6: Have you threatened to or tried to harm yourself or anyone else in the last year? 

In [454]:
var_name = 'hx_homeless_you_threatened'
axis_name = 'Attacked'
phq6 = binary_stat(var_name,axis_name,hdf)
phq6

,N,%
Attacked,,
no,18,94.737
yes,1,5.263
Total,19,100.000


### Question 7A: Do you have any legal stuff going on right now that may result in you being locked up, having to pay fines, or that make it more difficult to rent a place to live?

In [455]:
var_name = 'a_do_you_have_any_legal_st'
axis_name = 'Ongoing legal issues'
phq7a = binary_stat(var_name,axis_name,hdf)
phq7a

,N,%
Ongoing legal issues,,
no,14,73.684
yes,5,26.316
Total,19,100.000


### Question 7B: While experiencing homelessness, did you have any legal stuff going on that resulted in you being locked up, having to pay fines, or that made it more difficult to rent a place to live? 

In [456]:
var_name = 'experiencing_homeless_lockeup'
axis_name = 'Legal issues while homeless'
phq7b = binary_stat(var_name,axis_name,hdf)
phq7b

,N,%
Legal issues while homeless,,
no,11,57.895
yes,8,42.105
Total,19,100.000


### Question 8A: Does anybody force or trick you to do things that you do not want to do?

In [457]:
var_name = 'a_does_anybody_force_or_tr'
axis_name = 'Force/trick'
phq8a = binary_stat(var_name,axis_name,hdf)
phq8a

,N,%
Force/trick,,
no,18,100.0
Total,18,100.0


### Question 8B: While experiencing homelessness, did anybody force or trick you to do things that you do not want to do?

In [458]:
var_name = 'force_trick_while_homeless'
axis_name = 'Force/trick'
phq8b = binary_stat(var_name,axis_name,hdf)
phq8b

,N,%
Force/trick,,
no,13,72.222
yes,5,27.778
Total,18,100.000


### Question 9a: Do you ever do things that may be considered to be risky like exchange sex for money, run drugs for someone, have unprotected sex with someone you don’t know, share a needle, or anything like that?

In [459]:
var_name = 'a_do_you_ever_do_things_th'
axis_name = 'Risky activity'
phq9a = binary_stat(var_name,axis_name,hdf)
phq9a

,N,%
Risky activity,,
yes,13,72.222
no,5,27.778
Total,18,100.000


### Question 9b: While experiencing homelessness, do you ever do things that may be considered to be risky like exchange sex for money, run drugs for someone, have unprotected sex with someone you don’t know, share a needle, or anything like that?

In [460]:
var_name = 'while_homeless_riskythings'
axis_name = 'Risky activity while homeless'
phq9b = binary_stat(var_name,axis_name,hdf)
phq9b

,N,%
Risky activity while homeless,,
yes,13,72.222
no,5,27.778
Total,18,100.000


## 3C: Socialization and Daily Functioning

### Question 10: Is there any person, past landlord, business, bookie, dealer, or government group like the IRS that thinks you owe them money? 

In [461]:
var_name = 'hx_homelessness_any_person_past_l'
axis_name = 'Owe money'
phq10 = binary_stat(var_name,axis_name,hdf)
phq10

,N,%
Owe money,,
no,13,68.421
yes,6,31.579
Total,19,100.000


### Question 11: Do you get any money from the government, a pension, an inheritance, working under the table, a regular job, or anything like that? 

In [462]:
var_name = 'hx_homelessness_money_from'
axis_name = 'Money received'
phq11 = binary_stat(var_name,axis_name,hdf)
phq11

,N,%
Money received,,
no,12,63.158
yes,7,36.842
Total,19,100.000


### Question 12a: Do you have planned activities, other than just surviving, that make you feel happy and fulfilled? 

In [463]:
var_name = 'hx_homelessness_planned_activities'
axis_name = 'Fulfilling activities'
phq12a = binary_stat(var_name,axis_name,hdf)
phq12a

,N,%
Fulfilling activities,,
yes,13,68.421
no,6,31.579
Total,19,100.000


### If yes, what were the activities?

In [464]:
phq12c = pd.DataFrame(hdf['if_yes_print']).set_axis(['activities'], axis=1)
phq12c[~phq12c.isnull().any(axis=1)]

,activities
5,Music and pets
7,taking care of daughter
13,"Drawing, cooking"
14,talking to husband
16,Dating new people
17,playing video games
22,spending time with family
25,Being a mother
32,Twelve step
33,Golf


### Question 12b: While experiencing homelessness, did you have planned activities, other than just surviving, that make you feel happy and fulfilled? 

In [465]:
var_name = 'hx_homelessness_activities'
axis_name = 'Fulfilling activities while homeless'
phq12b = binary_stat(var_name,axis_name,hdf)
phq12b

,N,%
Fulfilling activities while homeless,,
no,15,78.947
yes,4,21.053
Total,19,100.000


### Question 13a: Are you currently able to take care of basic needs like bathing, changing clothes, using a restroom, getting food and clean water and other things like that? 

In [466]:
var_name = 'hx_homeless_able_to'
axis_name = 'Basic needs'
phq13a = binary_stat(var_name,axis_name,hdf)
phq13a

,N,%
Basic needs,,
yes,19,100.0
Total,19,100.0


### Question 13b: While experiencing homelessness, were you able to take care of basic needs like bathing, changing clothes, using a restroom, getting food and clean water and other things like that? 

In [467]:
var_name = 'b_while_experiencing_homel'
axis_name = 'Basic needs while homeless'
phq13b = binary_stat(var_name,axis_name,hdf)
phq13b

,N,%
Basic needs while homeless,,
no,12,63.158
yes,7,36.842
Total,19,100.000


### Question 14: Was your prior homelessness in any way caused by a relationship that broke down, an unhealthy or abusive relationship, or because family or friends caused you to become evicted?

In [468]:
var_name = 'was_your_prior_homelessnes'
axis_name = 'Related to relationship'
phq14 = binary_stat(var_name,axis_name,hdf)
phq14

,N,%
Related to relationship,,
yes,14,73.684
no,5,26.316
Total,19,100.000


## 4D: Wellness 

### Question 15: Have you ever had to leave an apartment, shelter program, or other place you were staying because of your physical health? 

In [469]:
var_name = 'hx_homesslessnes_leave_accomo'
axis_name = 'Housing issue due to health'
phq15 = binary_stat(var_name,axis_name,hdf)
phq15

,N,%
Housing issue due to health,,
no,16,84.211
yes,3,15.789
Total,19,100.000


### Question 16: Do you have any chronic health issues with your liver, kidneys, stomach, lungs or heart? 

In [470]:
var_name = 'hx_homelessness_chronic_he'
axis_name = 'Chronic health conditions'
phq16 = binary_stat(var_name,axis_name,hdf)
phq16

,N,%
Chronic health conditions,,
no,10,52.632
yes,9,47.368
Total,19,100.000


### Question 17a: When you are sick or not feeling well, do you avoid getting help?

In [471]:
var_name = 'avoidhelp_hx_homelessness_sick'
axis_name = 'Avoid seeking help'
phq17 = binary_stat(var_name,axis_name,hdf)
phq17

,N,%
Avoid seeking help,,
yes,11,57.895
no,8,42.105
Total,19,100.000


### Question 17b: When you were experiencing homelessness, when you were sick or not feeling well, did you avoid getting help? 

In [472]:
var_name = 'b_when_you_were_experienci'
axis_name = 'Avoid seeking help while homeless'
phq17b = binary_stat(var_name,axis_name,hdf)
phq17b

,N,%
Avoid seeking help while homeless,,
yes,12,63.158
no,7,36.842
Total,19,100.000


### Question 18: Has drinking or drug use led you to being kicked out of an apartment or program where you were staying in the past? 

In [473]:
var_name = 'hx_homelessness_drinking_drug_use'
axis_name = 'Substance use affect housing'
phq18 = binary_stat(var_name,axis_name,hdf)
phq18

,N,%
Substance use affect housing,,
yes,14,73.684
no,5,26.316
Total,19,100.000


### Question 19a: Are there any medications that a doctor said you should be taking that, for whatever reason, you are not taking? 
*cannot find variable for this*

### Question 19b: When you were experiencing homelessness, were there any medications that a doctor said you should be taking that, for whatever reason, you are not taking?

In [474]:
var_name = 'hx_homelessness_nottaking_meds'
axis_name = 'Not taking medications'
phq19b = binary_stat(var_name,axis_name,hdf)
phq19b

,N,%
Not taking medications,,
yes,11,57.895
no,8,42.105
Total,19,100.000


### Question 20: Are there any medications like painkillers that you don’t take the way the doctor prescribed or where you sell the medication? 

In [475]:
var_name = 'hx_homelessness_abuse_medications'
axis_name = 'Medication misuse'
phq20 = binary_stat(var_name,axis_name,hdf)
phq20

,N,%
Medication misuse,,
no,15,78.947
yes,4,21.053
Total,19,100.000


### Question 21: Was your prior period of homelessness caused by an experience of emotional, physical, psychological, sexual, or other type of abuse, or by any other trauma you have experienced?

In [476]:
var_name = 'hx_homelessness_cause'
axis_name = 'Abuse/trauma'
phq21 = binary_stat(var_name,axis_name,hdf)
phq21

,N,%
Abuse/trauma,,
yes,10,52.632
no,9,47.368
Total,19,100.000


***

# Health Leads Screening Toolkit

### Question 1: In the last 12 months*, did you ever eat less than you felt you should because there wasn’t enough money for food? 

In [477]:
var_name = 'in_the_last_12_months_didyes'
axis_name = 'Eat less'
hlq1 = binary_stat2(var_name,axis_name,hdf)
hlq1

,N,%
Eat less,,
yes,36,72.0
no,14,28.0
Total,50,100.0


### Question 2: Do problems getting child care make it difficult for you to work or study? (leave blank if you do not have children) 

In [478]:
var_name = 'problems_with_childcare_le'
axis_name = 'Problems with child care'
hlq2 = binary_stat3(var_name,axis_name,hdf)
hlq2

,N,%
Problems with child care,,
no,32,64.0
does not apply,12,24.0
yes,6,12.0
Total,50,100.0


### Question 3: In the last 12 months, have you needed to see a doctor, but could not because of cost? 

In [479]:
var_name = 'past_12_months_unable_to_s'
axis_name = 'Unable to see doctor'
hlq3 = binary_stat2(var_name,axis_name,hdf)
hlq3

,N,%
Unable to see doctor,,
no,43,86.0
yes,7,14.0
Total,50,100.0


### Question 4: In the last 12 months, have you ever had to go without health care because you didn’t have a way to get there? 

In [480]:
var_name = 'past_12_months_healthcare'
axis_name = 'No transport to healthcare'
hlq4 = binary_stat2(var_name,axis_name,hdf)
hlq4

,N,%
No transport to healthcare,,
no,28,57.143
yes,21,42.857
Total,49,100.000


### Question 5:	Do you ever need help reading hospital materials? 
*assuming 0 no, 1 yes*

In [481]:
var_name = 'help_reading_hospital_mate'
axis_name = 'Help needed reading hospital materials'
hlq5 = binary_stat(var_name,axis_name,hdf)
hlq5

,N,%
Help needed reading hospital materials,,
no,39,78.0
yes,11,22.0
Total,50,100.0


### Question 6: Do you often feel that you lack companionship?

In [482]:
var_name = 'lack_of_companionship'
axis_name = 'Lack of companionship'
hlq6 = binary_stat(var_name,axis_name,hdf)
hlq6

,N,%
Lack of companionship,,
yes,33,66.0
no,17,34.0
Total,50,100.0


### Is patient currently housed?

In [483]:
var_name = 'is_patient_currently_house'
axis_name = 'currently housed'
hlqh = binary_stat(var_name,axis_name,hdf)
hlqh

,N,%
currently housed,,
no,32,64.0
yes,18,36.0
Total,50,100.0


### *Only if participant is housed*
*18 identified as unhoused, 19 responding*

### Question 7:	In the last 12 months, has the electric, gas, oil, or water company threatened to shut off your services in your home?

In [484]:
var_name = 'if_yes_to_housed_past_12_m'
axis_name = 'services threatened'
hlq7 = binary_stat(var_name,axis_name,hdf)
hlq7

,N,%
services threatened,,
no,13,68.421
yes,6,31.579
Total,19,100.000


### Question 8:	Are you worried that in the next 2 months, you may not have stable housing?

In [485]:
var_name = 'are_you_worried_that_in_th'
axis_name = 'worried'
hlq8 = binary_stat(var_name,axis_name,hdf)
hlq8

,N,%
worried,,
yes,11,57.895
no,8,42.105
Total,19,100.000


***

***

***

# Modified analysis

# Section 3: Patients with a past homelessness experience who are currently housed

# Risks HOUSED

In [513]:
def times_count(var_name, df):
    qdata = pd.DataFrame(df[var_name].describe()).round(2).rename(columns={var_name: ' '}, index={'count':'N (responses)'})
    display(qdata)
    print('mode: '+ str(df[var_name].mode()[0]))
    print('')
    print(hdf[var_name].value_counts())

### 4. In the last six months, how many times have you:

**Received health care at an emergency department/room?**

In [487]:
var_name = 'whilehomeless_received_health_care_at'
times_count(var_name,hdf)

,
N (responses),19.00
mean,1.53
std,2.91
min,0.00
25%,0.00
50%,1.00
75%,2.00
max,13.00


mode: 0.0

whilehomeless_received_health_care_at
0.0     8
2.0     6
1.0     4
13.0    1
Name: count, dtype: int64


No: 8<br>
Yes: 11<br>
Total: 19<br>

**Taken an ambulance to the hospital?**

In [488]:
var_name = 'whilehomeless_taken_an_ambulance_to_th'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.89
std,1.63
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,5.00


mode: 0.0

whilehomeless_taken_an_ambulance_to_th
0.0    12
1.0     4
5.0     2
3.0     1
Name: count, dtype: int64


No: 12<br>
Yes: 7<br>
Total: 19<br>

**Been hospitalized as an inpatient?**

In [489]:
var_name = 'homeless_hospitalized'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.53
std,0.90
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,3.00


mode: 0.0

homeless_hospitalized
0.0    13
1.0     3
2.0     2
3.0     1
Name: count, dtype: int64


No: 13<br>
Yes: 6<br>
Total: 19<br>

**Used a crisis service, including sexual assault crisis, mental health crisis, family/intimate violence, distress centers and suicide prevention hotlines?**

In [490]:
var_name = 'while_homeless_crisesservice'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.37
std,0.68
min,0.00
25%,0.00
50%,0.00
75%,0.50
max,2.00


mode: 0.0

while_homeless_crisesservice
0.0    14
1.0     3
2.0     2
Name: count, dtype: int64


No: 14<br>
Yes: 5<br>
Total: 19<br>

**Used a crisis service, including sexual assault crisis, mental health crisis, family/intimate violence, distress centers and suicide prevention hotlines?**

In [491]:
var_name = 'while_homeless_crisesservice'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.37
std,0.68
min,0.00
25%,0.00
50%,0.00
75%,0.50
max,2.00


mode: 0.0

while_homeless_crisesservice
0.0    14
1.0     3
2.0     2
Name: count, dtype: int64


No: 14<br>
Yes: 5<br>
Total: 19<br>

**Talked to police because you witnessed a crime, were the victim of a crime, or the alleged perpetrator of a crime or because the police told you that you must move along?**

In [492]:
var_name = 'homeless_talked_to_police'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.42
std,1.02
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,3.00


mode: 0.0

homeless_talked_to_police
0.0    16
3.0     2
2.0     1
Name: count, dtype: int64


No: 16<br>
Yes: 3<br>
Total: 19<br>

**Stayed one or more nights in a holding cell, jail or prison, whether that was a short-term stay like the drunk tank, a longer stay for a more serious offence, or anything in between?**

In [493]:
var_name = 'homeless_stayed_one_or_more_night'
times_count(var_name,hdf)

,
N (responses),19.00
mean,0.26
std,0.56
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,2.00


mode: 0.0

homeless_stayed_one_or_more_night
0.0    15
1.0     3
2.0     1
Name: count, dtype: int64


No: 15<br>
Yes: 4<br>
Total: 19<br>

# Section 2: Patients currently experiencing homelessness

# Risks UNHOUSED

### 4. In the last six months how many times have you: 

**Received care at an emergency department?**

In [514]:
var_name = 'a_received_health_care_at'
times_count(var_name,hdf)

,
N (responses),31.00
mean,1.74
std,2.46
min,0.00
25%,0.00
50%,1.00
75%,3.00
max,10.00


mode: 0.0

a_received_health_care_at
0.0     13
1.0      6
3.0      5
2.0      3
4.0      2
10.0     1
9.0      1
Name: count, dtype: int64


No: 13<br>
Yes: 18<br>
Total: 31<br>

**Taken an ambulance to hospital?**

In [515]:
var_name = 'b_taken_an_ambulance_to_th'
times_count(var_name,hdf)

,
N (responses),31.00
mean,1.52
std,2.13
min,0.00
25%,0.00
50%,1.00
75%,2.00
max,10.00


mode: 0.0

b_taken_an_ambulance_to_th
0.0     12
1.0      8
2.0      5
3.0      3
6.0      1
10.0     1
4.0      1
Name: count, dtype: int64


No: 12<br>
Yes: 19<br>
Total: 31<br>

**Been hospitalized as an inpatient?**

In [516]:
var_name = 'c_been_hospitalized_as_an'
times_count(var_name,hdf)

,
N (responses),31.00
mean,0.74
std,1.26
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,6.00


mode: 0.0

c_been_hospitalized_as_an
0.0    19
2.0     6
1.0     5
6.0     1
Name: count, dtype: int64


No: 19<br>
Yes: 12<br>
Total: 31<br>

**Used a crisis service, including sexual assault crisis, mental health crisis, family/intimate violence, distress centers and suicide prevention hotlines?**

In [517]:
var_name = 'd_used_a_crisis_service_in'
times_count(var_name,hdf)

,
N (responses),31.00
mean,0.32
std,0.83
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,4.00


mode: 0.0

d_used_a_crisis_service_in
0.0    25
1.0     4
2.0     1
4.0     1
Name: count, dtype: int64


No: 25<br>
Yes: 6<br>
Total: 31<br>

**Talked to police because you witnessed a crime, were the victim of a crime, or the alleged perpetrator of a crime or because the police told you that you must move along?**

In [518]:
var_name = 'e_talked_to_police_because'
times_count(var_name,hdf)

,
N (responses),31.00
mean,0.58
std,1.03
min,0.00
25%,0.00
50%,0.00
75%,1.00
max,4.00


mode: 0.0

e_talked_to_police_because
0.0    21
1.0     5
2.0     3
3.0     1
4.0     1
Name: count, dtype: int64


No: 21<br>
Yes: 10<br>
Total: 31<br>

**Stayed one or more nights in a holding cell, jail or prison, whether that was a short-term stay like the drunk tank, a longer stay for a more serious offence, or anything in between?**

In [519]:
var_name = 'f_stayed_one_or_more_night'
times_count(var_name,hdf)

,
N (responses),31.00
mean,0.48
std,1.81
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,10.00


mode: 0.0

f_stayed_one_or_more_night
0.0     25
1.0      5
10.0     1
Name: count, dtype: int64


No: 25<br>
Yes: 6<br>
Total: 31<br>

***

## 3B: Vulnerability Index Section MODIFIED

Calculate a median and IQR for the following variables:<br><br>
Vulnerability Index Section 1, Question 2: How long has it been since you lived in permanent stable housing (months)?<br>

Vulnerability Index Section 1, Question 3: In the last three years, how many times have you been homeless?<br>

Vulnerability Index Section 2, Question 4:  In the last 6 months how many times have you received care at an emergency department?<br>

Of note, you split folks up into 2 groups (currently homeless and currently housed) so I'd want this for both groups as well as the overall median/IQR if possible<br>


### Question 2: How long has it been since you lived in permanent stable housing (months)? 

In [494]:
hq2 = pd.DataFrame(hdf['time_permanenthousing'].describe()).round(2).rename(columns={'time_permanenthousing': 'months since permanent housing'}, index={'count':'N (responses)'})
display(hq2)
print('overall median: ' + str(np.median(hdf['time_permanenthousing'][~np.isnan(hdf['time_permanenthousing'])])))
print('overall IQR: ' + str(hq2.loc['75%']-hq2.loc['25%']))

,months since permanent housing
N (responses),31.00
mean,61.26
std,87.45
min,1.00
25%,6.50
50%,24.00
75%,66.00
max,360.00


overall median: 24.0
overall IQR: months since permanent housing    59.5
dtype: float64


### Question 3: In the last three years, how many times have you been homeless? 

In [495]:
hq3 = pd.DataFrame(hdf['no_of_times_homeless_three_yrs'].describe()).round(2).rename(columns={'no_of_times_homeless_three_yrs': 'number of times homeless past three years'}, index={'count':'N (responses)'})
display(hq3)
print('overall median: ' + str(np.median(hdf['no_of_times_homeless_three_yrs'][~np.isnan(hdf['no_of_times_homeless_three_yrs'])])))
print('overall IQR: ' + str(hq3.loc['75%']-hq3.loc['25%']))

,number of times homeless past three years
N (responses),31.00
mean,2.42
std,3.39
min,1.00
25%,1.00
50%,2.00
75%,2.50
max,20.00


overall median: 2.0
overall IQR: number of times homeless past three years    1.5
dtype: float64


### Question 4: In the last 6 months how many times have you: 

**Received care at an emergency department?**

In [497]:
hq4a = pd.DataFrame(hdf['a_received_health_care_at'].describe()).round(2).rename(columns={'a_received_health_care_at': ' '}, index={'count':'N (responses)'})
display(hq4a)
print('overall median: ' + str(np.median(hdf['a_received_health_care_at'][~np.isnan(hdf['a_received_health_care_at'])])))
print('overall IQR: ' + str(hq4a.loc['75%']-hq4a.loc['25%']))

,
N (responses),31.00
mean,1.74
std,2.46
min,0.00
25%,0.00
50%,1.00
75%,3.00
max,10.00


overall median: 1.0
overall IQR:     3.0
dtype: float64
